# Drift-diffusion models (DDMs)

_Kira Dusterwald, November 2022, based loosely on https://compneuro.neuromatch.io/tutorials/W2D2_LinearSystems/student/W2D2_Tutorial3.html and https://gist.github.com/mwaskom/f8c118a956c549ecea7c. There are some great explanations and some derivations in https://academic.oup.com/mit-press-scholarship-online/book/29407/chapter/244818296 (also available as a textbook in Gatsby!)._

DDMs are common perceptual decision-making models pioneered by Ratcliff. They have been used to explain the mechanisms of (2-alternative forced choice*) decision-making, and to account for outcomes like direction of choice and reaction time. 

In short, DDMs model the accumulation of evidence towards bounds over time in an uncertain environment.

The purpose of this notebook / assignment is to provide a general framework for concepts you will encounter in the perceptual decision-making neuroscience literature, and an intuition of how to implement a drift-diffusion model yourself.

*at least for the simple versions we discuss here, but there are ways to extend the concepts

In [1]:
# prerequisites and imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:
# plotting helper function

def plotting(trials,decthreshold=10):
    f, ax = plt.subplots()
    if hasattr(trials[0], '__len__'):
        for t in trials:
            ax.plot(t,alpha=0.5)
    else:
        ax.plot(trials,alpha=0.5)
    ax.axhline(0, c=".7")
    ax.axhline(decthreshold, c=".2")
    ax.axhline(-decthreshold, c=".2")
    ax.set(ylim=(-decthreshold - 2, decthreshold + 2))
    ax.set_xlabel('time')
    ax.set_ylabel('evidence')
    plt.show()
    return

## Signal detection theory (SDT)

SDT has historically been employed to understand decision-making given uncertainty, literally detecting the "signal" from the "noise", and it's where we start our foray into decision-making models.

Consider a scenario in which a doctor needs to look at a chest x-ray and comment on whether or not the patient has tuberculosis (TB). There are many variables that affect x-ray quality and the manifestation of disease on an x-ray, which provide _uncertainty_. The doctor answers, producing several outcomes, which I include because you'll likely see them used in papers:
* True hit (true positive, i.e. has TB and x-ray = TB)
* False alarm (false positive, i.e. does not have TB but x-ray = TB)
* Correct rejection (true negative, i.e. does not have TB and x-ray $\neq$ TB)
* Miss (false negative, i.e. has TB but x-ray $\neq$ TB)

Perceptually, we separate this process into an information acquisition stage, which can be corrupted by **external noise** like imaging quality, and a criterion or response stage, which might be corrupted by a doctor's **prior** or bias. For example, if a doctor believes that TB is dangerous and prevalent, they might think it is better to start treatment, and so err on diagnosing TB in conditions of uncertain information.

The information acquisition part is sometimes called a "generative process" since it generates an **internal representation** of the variable of interest in the x-ray in terms of neural spikes. This, of course, is also party to **internal noise** because neurons do not provide perfect fidelity. We assume that while an individual neuron might not code the doctor's decision about TB, there is some **internal response** generated from the internal representation, i.e. an aspect of neural activity that reflects the doctor's decision.

We will make more of these concepts concrete as we go along, but the terminology is useful to take on.

**PROBLEM**
1. You could imagine a fun test between doctors when we ask them to give a diagnosis as quickly as possible. Then the doctor will make some rushed decisions. What might happen to accuracy (i.e. proportion of correct responses (true hits + correct rejections)) in this scenario compared to on a normal day?

2. Can SDT account for the accuracy-response time trade off?

 ## Time and diffusion

What's significantly left out in the SDT framework is an incorporation of a key variable: time. This matters for several reasons. First, when some variable is loaded in working memory, there should be some lag between stimulus and response of the subject. Noise over time doesn't stay where it is, and it's important to incorporate how it moves, or diffuses. 

As an example, let's simulate a random walk (Brownian motion). Say you are watching a particle that starts at a position $x=0$. 

**PROBLEM** 

3. First, at each time step, simulate and plot over time what happens when your particle goes up or down $x$ by 1 with equal probability (like a coin flip). 

In [ ]:
# time array and position array
t_end = 100
times = np.arange(t_end)
x = [0]
t = 1

# loop through time
while t < t_end:
    evidence = ## YOUR CODE HERE
    x.append() ## YOUR CODE HERE
    t += 1

plotting(np.array(x))

**PROBLEM**

4. Now subsitute your rule of how the position shifts at each time step for a Gaussian random variable (start with mean $\mu = 0$ and variance $\sigma^2 = 1$, that is $x \sim \mathcal{N}(0,1)$). What happens when you change the mean and variance?

5. Simulate multiple trajectories. What happens when you change the time horizon?

6. What is the expected value of the particle's position as a function of time? What is the empiric average? What is the expected variance as a function of time? Plot the empiric mean and variance over time.

In [18]:
def rand_walk(mean,variance):
    
    return np.array(x)

In [ ]:
# implement for multiple trials
ntrials = 1000
mymean = 0
myvar = 1

array_trials = np.array([rand_walk(mymean,myvar) for _ in range(ntrials)])
plotting(array_trials)

In [ ]:
empiric_mu = ## YOUR CODE HERE
empiric_var = ## YOUR CODE HERE

plt.plot(empiric_mu)
plt.ylabel('$\mu$')
plt.xlabel('time')
plt.ylim([-1-mymean,1+mymean])
plt.show()

plt.plot(empiric_var)
plt.ylabel('$\sigma^2$')
plt.xlabel('time')
plt.show()

**BONUS**

7. Derive the expected mean and variance after $\tau$ time steps, given $\mu$ and $\sigma^2$ fixed for each time step.

### Adding a bound to signal evidence accumulation

**PROBLEM**

8. Add _decision bounds_ to your code by copying the single trial function to the cell below. I.e. When an upwards or downwards random walk gets to a set bound, terminate that trajectory. Then plot the reaction times as a histogram. What does the distribution of the RT histogram look like? Play with the parameters a little to get a feel for how diffusion affects RTs.

9. How would you calculate the proportion of trials terminating at the "top" and "bottom" bounds respectively?

In [ ]:
def rand_walk_bound(mean,variance,bound):
    ## YOUR CODE HERE
    
    return np.array(x)

# implement for multiple trials
ntrials = 1000
mymean = 0
myvar = 2
mybound = 10

array_trials_bound = np.array([rand_walk_bound(mymean,myvar,mybound) for _ in range(ntrials)])
plotting(array_trials_bound,decthreshold=mybound)

# calculate reaction times
reaction_times =  ## YOUR CODE HERE

plt.hist(reaction_times)
plt.xlabel('time bins')
plt.ylabel('proportion')
plt.show()

In [ ]:
# calculate proportions
prop_top =  ## YOUR CODE HERE
prop_bottom =  ## YOUR CODE HERE

print('proportion of top bound: ' + str(np.round(prop_top,2)))
print('proportion of bottom bound: ' + str(np.round(prop_bottom,2)))

## How does this link to DDM?

In a DDM sense, momentary evidence is corrupted by noise which is integrated over time. This causes the "decision variable" to move around in a Brownian way, until it hits or gets "absorbed" by some bound, at which point that bound wins and the decision is made. The time this takes gives one the reaction time. The amount of noise (variance) might influence when a decision is made or not.

Using diffusion like this is a popular way to model working memory: hanging onto a variable imperfectly.

We would like to include some deterministic _bias_ in our model of decision-making, which can help us weight decisions but decays over time. To do so, incorporate a *drift* term: a term that slowly tends towards some value. This you can think of in the dynamical systems sense, so that sensory information from previous time steps is weighted less over time.

We can think of a simple linear differential equation in continuous time as $\frac{d}{dt} x =  (\lambda-1)(x - x_\infty)$ for some scalar $\lambda \in \mathbb{R}$ and constant $x_\infty$. However so that we can simulate it, we discretise it*:

\begin{equation*}
x_{t+1} = \lambda (x_t - x_\infty) + x_\infty
\end{equation*}

**PROBLEM**

10. Solve this DE. Assume $x_0$ is the value of $x$ when $t=0$.

11. Simulate this DE using either the closed form solution, discretised approach, or a numerical approach (e.g. Euler's; speak to a TA if you have not heard of this before!).

12. How does the value of $\lambda$ affect the solution? When is it stable or not?

*Slightly informal, but basically this is a Euler approach with $dt=1$. :)

In [ ]:
x_lam = 0.9 # e^(lambda-1)
x_0 = 2
x_infinity = 1
t_end = 100
times = np.linspace(0,t_end,1000)

x = ## YOUR CODE HERE

plt.plot(times, x)
plt.xlabel('time')
plt.show()

## Full DDM!

The full DDM is just a combination of the drift and diffusion terms, that is:
\begin{equation*}
x_{t+1} = \lambda (x_t - x_\infty) + x_\infty + \kappa \xi
\end{equation*},

where $\kappa$ can be set and $\xi \sim \mathcal{N} (\mu,\sigma)$. Usually, $\mu =0$.

**PROBLEM**

13. Implement full DDM. Simulate several trials and explore.

14. What happens to the empirical mean and variance in full DDM over time? Plot this. Play with the parameters. What does the variance seem to depend on?

In [ ]:
lam = 0.9
x_inf = 1
t_end = 100
trials = 100

## YOUR CODE HERE
array_trials = ##

plotting(array_trials)

From the above, you should see that the inclusion of the deterministic term provides a nice hack to balance the continually growing variance from the stochastic diffusion process.

**BONUS**

15. Calculate the analytical variance of the full DDM at equilibrium.

## Fitting behavioural data

So we've now walked through the model, but we would also like to be able to apply it to real data. There are a few general approaches for this type of model-fitting exercise, but these basically boil down to simulating the DDM many times with different parameter values until you optimise a criterion (e.g. maximising the likelihood of the data). You can employ a non-linear search algorithm to efficiently work through the parameter space. This type of approach may be needed for complicated models or when you really want to fit some more nuanced dynamic, e.g. the reaction time distribution. 

However, for a simple fit of the accuracy and mean reaction times, there is a closed form solution! Let's assume we just have the diffusion part for this exercise, i.e. $x_{t+1} = x_t + \xi$, and let the top bound be given by $\beta$ and the bottom by $-\beta$. This is still an okay model for working memory, we just do not have the decay term for the sensory evidence mean. Then we can show that:

\begin{equation*}
P_{top} = \frac{1}{1+\exp\Big[-\frac{2\mu \beta}{\sigma^2}\Big]}
\end{equation*}

**BONUS**

16. Derive this expression.

17. Simulate the equation for different values of $\mu$. What about the bounds and the variance? (see code below)

18. Derive the expression to calculate the mean reaction time for when you hit the bound,

\begin{equation*}
RT_{mean} = \frac{\beta}{\mu}\tanh[\mu\beta]
\end{equation*}

19. Plot the expected mean RT for different values of $\mu$. What happens to this expression when $\mu \rightarrow 0$? Take the limit and derive the solution. You can think of this as a case for which there is no sensory evidence, so it is important to cover.

20. What happens when $\mu$ is really large? RTs cannot be instantaneous! To fix this problem, people add or fit a fixed delay constant term $\tau$ to the mean RT equation.


In [ ]:
# q 17
beta = 1
mu = np.linspace(0, 10, 100)
var = 1
p = ## YOUR CODE HERE

f, ax = plt.subplots(figsize=(4, 4))
ax.plot(mu, p)
ax.set(xlabel="$\mu$", ylabel="$P_{correct}$", ylim=(.5, 1.1))

# q 19
rt = ## YOUR CODE HERE
f, ax = plt.subplots(figsize=(4, 4))
ax.plot(mu, rt)
ax.set(xlabel="$\mu$", ylabel="$RT_{mean}$")

## Fitting behavioural data continued

We can use the framework above to fit data from a motion coherence task: https://eutils.ncbi.nlm.nih.gov/entrez/eutils/elink.fcgi?dbfrom=pubmed&id=8257671&retmode=ref&cmd=prlinks. 

This is left as an exercise to the reader: use the parameters in Fig 3. :)